In [1]:
""" 제 2고지 자연스러운 코드로 """

' 제 2고지 자연스러운 코드로 '

In [2]:
""" STEP15. 복잡한 계산 그래프 ( 이론 편 ) """

' STEP15. 복잡한 계산 그래프 ( 이론 편 ) '

In [3]:
""" STEP15. 복잡한 계산 그래프 ( 구현 편 ) """


import numpy as np

# ndarray 인스턴스만 취급하고록 바꿈 ( 다른게 들어오면 오류 )

# Variable이라는 상자 생성
class Variable:
    def __init__(self, data): # 생성자
        # 입력 데이터가 None이 아닌 경우, 
        # 입력 데이터의 타입이 np.ndarray인지 확인
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(
                    '{}은(는) 지원하지 않습니다.'.format(type(data)))
         
        self.data = data # 변수의 데이터를 입력 데이터로 설정

        # 변수의 기울기 초기화
        self.grad = None # 미분값 저장하기 위한 변수

        # 변수를 생성한 함수(연산) 초기화
        self.creator = None # 연산을 나타내는 객체

        self.generation = 0 # 세대 수를 기록하는 변수

    # 해당 변수가 어떤 함수에 의해 만들어졌는지를 저장
    def set_creator(self, func):
        self.creator = func
        # 세대를 기록한다 ( 부모 세대 + 1)
        self.generation = func.generation + 1

    # 역전파를 자동화 할 수 있도록 새로운 메서드 생성
    # 반복문을 이용한 구현
    def backward(self):
        # y.grad = np.array(1.0) 생략을 위한 if문
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop() # 함수를 가져온다.
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys) # 함수 f의 역전파 호출 ( 리스트 언팩 )
            
            # gxs가 튜플이 아니라면 튜플로 변환
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            # 역전파로 전파되는 미분값을 Variable인스턴스 변수 grad에 저장
            for x, gx in zip(f.inputs, gxs): # gxs와 f.inputs는 대응
                if x.grad is None:
                    x.grad = gx
                else:
                    # x.grad += gx <- 문제 발생 ( 부록 A )
                    x.grad = x.grad + gx
                    

                if x.creator is not None:
                    add_func(x.creator) # 수정전 : funcs.append(x.creator)

            # if x.creator is not None:
            #     # 하나 앞의 함수를 리스트에 추가한다.
            #     funcs.append(x.creator)

    # Variable의 인스턴스 재사용시 문제 해결을 위해
    # Variable 클래스에 미분값 초기화하는 메서드 생성
    def cleargrad(self):
        self.grad = None

In [4]:
# 주어진 입력을 NumPy 배열로 변환하는 함수
def as_array(x):
    if np.isscalar(x):  # 입력이 스칼라인지 확인
        return np.array(x)  # 스칼라인 경우, 배열로 변환하여 반환
    return x  # 스칼라가 아닌 경우, 그대로 반환

In [5]:
# Variable 인스턴스를 변수로 다룰 수 있는 함수를 Function클래스로 구현
class Function:
    # *ㅁㅁㅁ : 임의 개수의 인수 ( 가변길이 ) 를 건내 함수를 호출할 수 있음
    def __call__(self, *inputs):
        # 리스트 xs를 생성할 때, 리스트 내포 사용
        # 리스트의 각 원소 x에 대해 각각 데이터 ( x.data ) 를 꺼냄
        xs = [x.data for x in inputs]
        
        # forward 메서드에서 구체적인 계산을 함
        ys = self.forward(*xs) # 리스트 언팩 ( 원소를 낱개로 풀어서 전달 )

        if not isinstance(ys, tuple): # 튜플이 아닌 경우 추가 지원
            ys = (ys, )

        # ys의 각 원소에 대해 Variable 인스턴스 생성, outputs 리스트에 저장
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])

        # 각 output Variable 인스턴스의 creator를 현재 Function 객체로 설정
        for output in outputs:
            output.set_creator(self)
        
        self.inputs = inputs # 입력 저장
        self.outputs = outputs # 출력 저장

        # 리스트의 원소가 하나라면 첫 번째 원소를 반환함
        return outputs if len(outputs) > 1 else outputs[0]
    
    # 순전파
    def forward(self, xs):
        raise NotImplementedError()
    
    # 역전파
    def backward(self, gys):
        raise NotImplementedError()

In [6]:
# 두 개의 입력을 받아 덧셈 수행
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy

In [7]:
def add(x0, x1):
    return Add()(x0, x1)

In [8]:
# y = x²
class Square(Function):
    # 순전파
    def forward(self, x):
        y = x ** 2 # y = x²
        return y
    
    # 역전파
    def backward(self, gy): # gy = 출력쪽에 전해지는 미분값을 전달하는 역할
        x = self.inputs[0].data # 수정전 : x = self.input.data
        gx = 2 * x * gy #  y' = 2x
        return gx

In [9]:
def square(x):
    return Square()(x)

In [10]:
# y = eˣ
class Exp(Function):
    # 순전파
    def forward(self, x):
        y = np.exp(x) # 주어진 입력값에 대한 지수 함수를 계산하여 반환
        return y
    
    # 역전파
    def backward(self, gy):
        x = self.input.data
        """ 지수 함수의 도함수는 자기 자신을 유지하므로 
            입력값의 지수 함수 값에 gy를 곱함 """
        gx = np.exp(x) * gy
        return gx

In [11]:
def exp(x):
    return Exp()(x)

In [12]:
#  y = ((x²)² + (x²)²)
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0
